In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import torch.utils.data as utils
import librosa
import soundfile as sf
import time
import os
from torch.utils import data
from wavenet import Wavenet
from transformData import x_mu_law_encode,y_mu_law_encode,mu_law_decode,onehot,cateToSignal
from readDataset import Dataset

In [2]:
sampleSize=32000#the length of the sample size
quantization_channels=256
sample_rate=16000
dilations=[2**i for i in range(9)]*5  #idea from wavenet, have more receptive field
residualDim=128 #
skipDim=512
shapeoftest = 190500
filterSize=3
songnum=10
savemusic='./vsCorpus/notextr{}.wav'
resumefile='./model/testac' # name of checkpoint
lossname='testacloss.txt' # name of loss file
continueTrain=False # whether use checkpoint
pad = np.sum(dilations) # padding for dilate convolutional layers
lossrecord=[]  #list for record loss
#pad=0

    #            |----------------------------------------|     *residual*
    #            |                                        |
    #            |    |-- conv -- tanh --|                |
    # -> dilate -|----|                  * ----|-- 1x1 -- + -->	*input*
    #                 |-- conv -- sigm --|     |    ||
    #                                         1x1=residualDim
    #                                          |
    # ---------------------------------------> + ------------->	*skip=skipDim*
    image changed from https://github.com/vincentherrmann/pytorch-wavenet/blob/master/wavenet_model.py

In [3]:
#os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # use specific GPU

In [4]:
use_cuda = torch.cuda.is_available() # whether have available GPU
torch.manual_seed(1)
device = torch.device("cuda" if use_cuda else "cpu")
#device = 'cpu'
#torch.set_default_tensor_type('torch.cuda.FloatTensor') #set_default_tensor_type as cuda tensor
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {} 

In [5]:
params = {'batch_size': 1, 'shuffle': True, 'num_workers': 1}
training_set = Dataset(np.arange(0, songnum), np.arange(0, songnum), 'ccmixter2/x/', 'ccmixter2/y/')
validation_set = Dataset(np.arange(0, songnum), np.arange(0, songnum), 'ccmixter2/x/', 'ccmixter2/y/')
loadtr = data.DataLoader(training_set, **params)  # pytorch dataloader, more faster than mine
loadval = data.DataLoader(validation_set, **params)

In [6]:
model = Wavenet(pad,skipDim,quantization_channels,residualDim,dilations).cuda()
criterion = nn.CrossEntropyLoss()
#in wavenet paper, they said crossentropyloss is far better than MSELoss
#optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer = optim.Adam(model.parameters(), lr=1e-3,weight_decay=1e-5)
#use adam to train
#optimizer = optim.SGD(model.parameters(), lr = 0.1, momentum=0.9, weight_decay=1e-5)
#scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
#scheduler = MultiStepLR(optimizer, milestones=[20,40], gamma=0.1)

In [7]:
if continueTrain:# if continueTrain, the program will find the checkpoints
    if os.path.isfile(resumefile):
        print("=> loading checkpoint '{}'".format(resumefile))
        checkpoint = torch.load(resumefile)
        start_epoch = checkpoint['epoch']
        #best_prec1 = checkpoint['best_prec1']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> loaded checkpoint '{}' (epoch {})"
              .format(resumefile, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(resumefile))

In [ ]:
'''def val(xtrain,ytrain): #validation last 15 seconds of the audio.
    model.eval()
    start_time = time.time()
    with torch.no_grad():
        idx = np.arange(xtrain.shape[-1]-pad-10*sampleSize,xtrain.shape[-1]-pad-sampleSize,1000)
        np.random.shuffle(idx)
        data = xtrain[:,:,idx[0]-pad:pad+idx[0]+sampleSize].to(device)
        target = ytrain[:,idx[0]:idx[0]+sampleSize].to(device)
        output = model(data)
        pred = output.max(1, keepdim=True)[1]
        correct = pred.eq(target.view_as(pred)).sum().item() / pred.shape[-1]
        val_loss = criterion(output, target).item()
        print(correct,'accurate')
        print('\nval set:loss{:.4f}:, ({:.3f} sec/step)\n'.format(val_loss,time.time()-start_time))
        
        listofpred = []
        for ind in range(xtrain.shape[-1]-pad-10*sampleSize,xtrain.shape[-1]-pad-sampleSize,sampleSize):
            output = model(xtrain[:, :, ind - pad:ind + sampleSize + pad].to(device))
            pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
            listofpred.append(pred)
        ans = mu_law_decode(np.concatenate(listofpred))
        sf.write('./vsCorpus/notexval.wav', ans, sample_rate)
        print('val stored done',time.time() - start_time)'''
        

def test(xtrain,iloader):# testing data
    model.eval()
    start_time = time.time()
    with torch.no_grad():
        '''for iloader,(xtest,_) in enumerate(loadval):
            listofpred = []
            for ind in range(pad, xtest.shape[-1] - pad, sampleSize):
                output = model(xtest[:, :, ind - pad:ind + sampleSize + pad].to(device))
                pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
                listofpred.append(pred)
            ans = mu_law_decode(np.concatenate(listofpred))
            sf.write('./vsCorpus/notexte.wav', ans, sample_rate)'''

        listofpred=[]
        for ind in range(pad,xtrain.shape[-1]-pad,sampleSize):
            output = model(xtrain[:, :, ind-pad:ind+sampleSize+pad].to(device))
            pred = output.max(1, keepdim=True)[1].cpu().numpy().reshape(-1)
            listofpred.append(pred)
        ans = mu_law_decode(np.concatenate(listofpred))
        sf.write(savemusic.format(iloader), ans, sample_rate)
        print('test stored done',time.time() - start_time)
    
def train(epoch):#training data, the audio except for last 15 seconds
    model.train()
    for iloader,(xtrain,ytrain) in enumerate(loadtr):
        idx = np.arange(pad,xtrain.shape[-1]-pad-sampleSize,8000)
        np.random.shuffle(idx)#random the starting points
        lens = idx.shape[-1] // songnum
        idx = idx[:lens]
        for i, ind in enumerate(idx):
            start_time = time.time()
            data, target = xtrain[:,:,ind-pad:ind+sampleSize+pad].to(device), ytrain[:,ind:ind+sampleSize].to(device)
            output = model(data)
            loss = criterion(output, target)
            lossrecord.append(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print('Train Epoch: {} iloader:{} [{}/{} ({:.0f}%)] Loss:{:.6f}: , ({:.3f} sec/step)'.format(
                epoch, iloader, i, len(idx), 100. * i / len(idx), loss.item(), time.time() - start_time))
            if i % 100 == 0:
                with open("./lossRecord/"+lossname, "w") as f:
                    for s in lossrecord:
                        f.write(str(s) +"\n")
                print('write finish')

        #val(xtrain,ytrain)
        test(xtrain,iloader)
        state={'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()}
        torch.save(state, resumefile)

In [ ]:
for epoch in range(100000):
    train(epoch)

Train Epoch: 0 iloader:0 [0/38 (0%)] Loss:5.562781: , (2.165 sec/step)
write finish
Train Epoch: 0 iloader:0 [1/38 (3%)] Loss:5.234249: , (2.137 sec/step)
Train Epoch: 0 iloader:0 [2/38 (5%)] Loss:4.972779: , (2.139 sec/step)
Train Epoch: 0 iloader:0 [3/38 (8%)] Loss:4.724823: , (2.138 sec/step)
Train Epoch: 0 iloader:0 [4/38 (11%)] Loss:4.612024: , (2.137 sec/step)
Train Epoch: 0 iloader:0 [5/38 (13%)] Loss:4.449580: , (2.138 sec/step)
Train Epoch: 0 iloader:0 [6/38 (16%)] Loss:4.436759: , (2.140 sec/step)
Train Epoch: 0 iloader:0 [7/38 (18%)] Loss:4.376324: , (2.143 sec/step)
Train Epoch: 0 iloader:0 [8/38 (21%)] Loss:4.326145: , (2.139 sec/step)
Train Epoch: 0 iloader:0 [9/38 (24%)] Loss:4.310313: , (2.141 sec/step)
Train Epoch: 0 iloader:0 [10/38 (26%)] Loss:4.976738: , (2.141 sec/step)
Train Epoch: 0 iloader:0 [11/38 (29%)] Loss:4.272497: , (2.140 sec/step)
Train Epoch: 0 iloader:0 [12/38 (32%)] Loss:4.316669: , (2.139 sec/step)
Train Epoch: 0 iloader:0 [13/38 (34%)] Loss:4.238191

Train Epoch: 0 iloader:2 [32/42 (76%)] Loss:3.959912: , (2.145 sec/step)
Train Epoch: 0 iloader:2 [33/42 (79%)] Loss:3.814303: , (2.145 sec/step)
Train Epoch: 0 iloader:2 [34/42 (81%)] Loss:4.043190: , (2.146 sec/step)
Train Epoch: 0 iloader:2 [35/42 (83%)] Loss:4.124961: , (2.144 sec/step)
Train Epoch: 0 iloader:2 [36/42 (86%)] Loss:3.923800: , (2.148 sec/step)
Train Epoch: 0 iloader:2 [37/42 (88%)] Loss:3.963415: , (2.146 sec/step)
Train Epoch: 0 iloader:2 [38/42 (90%)] Loss:3.735707: , (2.146 sec/step)
Train Epoch: 0 iloader:2 [39/42 (93%)] Loss:4.270330: , (2.144 sec/step)
Train Epoch: 0 iloader:2 [40/42 (95%)] Loss:3.785260: , (2.146 sec/step)
Train Epoch: 0 iloader:2 [41/42 (98%)] Loss:3.899356: , (2.148 sec/step)
test stored done 71.52951955795288
Train Epoch: 0 iloader:3 [0/33 (0%)] Loss:4.232036: , (2.150 sec/step)
write finish
Train Epoch: 0 iloader:3 [1/33 (3%)] Loss:4.258659: , (2.152 sec/step)
Train Epoch: 0 iloader:3 [2/33 (6%)] Loss:4.174275: , (2.153 sec/step)
Train Epo

Train Epoch: 0 iloader:6 [11/43 (26%)] Loss:4.533360: , (2.145 sec/step)
Train Epoch: 0 iloader:6 [12/43 (28%)] Loss:4.502923: , (2.146 sec/step)
Train Epoch: 0 iloader:6 [13/43 (30%)] Loss:4.586398: , (2.159 sec/step)
Train Epoch: 0 iloader:6 [14/43 (33%)] Loss:4.424753: , (2.160 sec/step)
Train Epoch: 0 iloader:6 [15/43 (35%)] Loss:4.495053: , (2.145 sec/step)
Train Epoch: 0 iloader:6 [16/43 (37%)] Loss:3.709264: , (2.146 sec/step)
Train Epoch: 0 iloader:6 [17/43 (40%)] Loss:4.614162: , (2.144 sec/step)
Train Epoch: 0 iloader:6 [18/43 (42%)] Loss:4.209762: , (2.145 sec/step)
Train Epoch: 0 iloader:6 [19/43 (44%)] Loss:4.744404: , (2.144 sec/step)
Train Epoch: 0 iloader:6 [20/43 (47%)] Loss:4.173180: , (2.145 sec/step)
Train Epoch: 0 iloader:6 [21/43 (49%)] Loss:4.531165: , (2.146 sec/step)
Train Epoch: 0 iloader:6 [22/43 (51%)] Loss:3.981793: , (2.145 sec/step)
Train Epoch: 0 iloader:6 [23/43 (53%)] Loss:4.501549: , (2.146 sec/step)
Train Epoch: 0 iloader:6 [24/43 (56%)] Loss:4.48330

Train Epoch: 0 iloader:8 [41/53 (77%)] Loss:4.736802: , (2.163 sec/step)
Train Epoch: 0 iloader:8 [42/53 (79%)] Loss:3.455180: , (2.147 sec/step)
Train Epoch: 0 iloader:8 [43/53 (81%)] Loss:4.700556: , (2.148 sec/step)
Train Epoch: 0 iloader:8 [44/53 (83%)] Loss:4.785316: , (2.147 sec/step)
Train Epoch: 0 iloader:8 [45/53 (85%)] Loss:4.524364: , (2.144 sec/step)
Train Epoch: 0 iloader:8 [46/53 (87%)] Loss:4.358671: , (2.147 sec/step)
Train Epoch: 0 iloader:8 [47/53 (89%)] Loss:4.386470: , (2.145 sec/step)
Train Epoch: 0 iloader:8 [48/53 (91%)] Loss:3.461838: , (2.146 sec/step)
Train Epoch: 0 iloader:8 [49/53 (92%)] Loss:3.700259: , (2.145 sec/step)
Train Epoch: 0 iloader:8 [50/53 (94%)] Loss:3.826400: , (2.146 sec/step)
Train Epoch: 0 iloader:8 [51/53 (96%)] Loss:4.553010: , (2.144 sec/step)
Train Epoch: 0 iloader:8 [52/53 (98%)] Loss:3.950964: , (2.148 sec/step)
test stored done 89.00384616851807
Train Epoch: 0 iloader:9 [0/47 (0%)] Loss:4.536586: , (2.146 sec/step)
write finish
Train

Train Epoch: 1 iloader:1 [10/42 (24%)] Loss:4.167428: , (2.145 sec/step)
Train Epoch: 1 iloader:1 [11/42 (26%)] Loss:4.305487: , (2.145 sec/step)
Train Epoch: 1 iloader:1 [12/42 (29%)] Loss:4.194024: , (2.147 sec/step)
Train Epoch: 1 iloader:1 [13/42 (31%)] Loss:4.479056: , (2.144 sec/step)
Train Epoch: 1 iloader:1 [14/42 (33%)] Loss:4.306422: , (2.145 sec/step)
Train Epoch: 1 iloader:1 [15/42 (36%)] Loss:4.261561: , (2.145 sec/step)
Train Epoch: 1 iloader:1 [16/42 (38%)] Loss:4.049470: , (2.144 sec/step)
Train Epoch: 1 iloader:1 [17/42 (40%)] Loss:4.476519: , (2.145 sec/step)
Train Epoch: 1 iloader:1 [18/42 (43%)] Loss:3.765738: , (2.146 sec/step)
Train Epoch: 1 iloader:1 [19/42 (45%)] Loss:4.120128: , (2.147 sec/step)
Train Epoch: 1 iloader:1 [20/42 (48%)] Loss:3.743572: , (2.145 sec/step)
Train Epoch: 1 iloader:1 [21/42 (50%)] Loss:4.123576: , (2.146 sec/step)
Train Epoch: 1 iloader:1 [22/42 (52%)] Loss:4.045243: , (2.147 sec/step)
Train Epoch: 1 iloader:1 [23/42 (55%)] Loss:3.77841

...

Train Epoch: 16 iloader:0 [10/47 (21%)] Loss:4.783000: , (2.133 sec/step)
Train Epoch: 16 iloader:0 [11/47 (23%)] Loss:4.879055: , (2.129 sec/step)
Train Epoch: 16 iloader:0 [12/47 (26%)] Loss:3.168573: , (2.132 sec/step)
Train Epoch: 16 iloader:0 [13/47 (28%)] Loss:3.318919: , (2.131 sec/step)
Train Epoch: 16 iloader:0 [14/47 (30%)] Loss:4.511611: , (2.131 sec/step)
Train Epoch: 16 iloader:0 [15/47 (32%)] Loss:4.606092: , (2.133 sec/step)
Train Epoch: 16 iloader:0 [16/47 (34%)] Loss:4.390665: , (2.133 sec/step)
Train Epoch: 16 iloader:0 [17/47 (36%)] Loss:4.251681: , (2.133 sec/step)
Train Epoch: 16 iloader:0 [18/47 (38%)] Loss:4.297935: , (2.132 sec/step)
Train Epoch: 16 iloader:0 [19/47 (40%)] Loss:4.341986: , (2.132 sec/step)
Train Epoch: 16 iloader:0 [20/47 (43%)] Loss:4.470053: , (2.132 sec/step)
Train Epoch: 16 iloader:0 [21/47 (45%)] Loss:4.620088: , (2.132 sec/step)
Train Epoch: 16 iloader:0 [22/47 (47%)] Loss:4.395302: , (2.133 sec/step)
Train Epoch: 16 iloader:0 [23/47 (49%)

Train Epoch: 16 iloader:2 [36/38 (95%)] Loss:2.934159: , (2.132 sec/step)
Train Epoch: 16 iloader:2 [37/38 (97%)] Loss:2.635167: , (2.131 sec/step)
test stored done 64.06553792953491
Train Epoch: 16 iloader:3 [0/33 (0%)] Loss:5.399499: , (2.133 sec/step)
write finish
Train Epoch: 16 iloader:3 [1/33 (3%)] Loss:5.063404: , (2.133 sec/step)
Train Epoch: 16 iloader:3 [2/33 (6%)] Loss:5.293294: , (2.132 sec/step)
Train Epoch: 16 iloader:3 [3/33 (9%)] Loss:4.938806: , (2.133 sec/step)
Train Epoch: 16 iloader:3 [4/33 (12%)] Loss:3.403686: , (2.132 sec/step)
Train Epoch: 16 iloader:3 [5/33 (15%)] Loss:4.794552: , (2.131 sec/step)
Train Epoch: 16 iloader:3 [6/33 (18%)] Loss:4.719301: , (2.132 sec/step)
Train Epoch: 16 iloader:3 [7/33 (21%)] Loss:4.453926: , (2.132 sec/step)
Train Epoch: 16 iloader:3 [8/33 (24%)] Loss:4.371878: , (2.133 sec/step)
Train Epoch: 16 iloader:3 [9/33 (27%)] Loss:4.241383: , (2.130 sec/step)
Train Epoch: 16 iloader:3 [10/33 (30%)] Loss:4.184318: , (2.132 sec/step)
Trai

Train Epoch: 16 iloader:6 [11/42 (26%)] Loss:4.641611: , (2.132 sec/step)
Train Epoch: 16 iloader:6 [12/42 (29%)] Loss:3.724872: , (2.131 sec/step)
Train Epoch: 16 iloader:6 [13/42 (31%)] Loss:3.383642: , (2.133 sec/step)
Train Epoch: 16 iloader:6 [14/42 (33%)] Loss:4.474443: , (2.131 sec/step)
Train Epoch: 16 iloader:6 [15/42 (36%)] Loss:2.967821: , (2.132 sec/step)
Train Epoch: 16 iloader:6 [16/42 (38%)] Loss:4.671126: , (2.134 sec/step)
Train Epoch: 16 iloader:6 [17/42 (40%)] Loss:4.693438: , (2.133 sec/step)
Train Epoch: 16 iloader:6 [18/42 (43%)] Loss:3.893876: , (2.132 sec/step)
Train Epoch: 16 iloader:6 [19/42 (45%)] Loss:4.275182: , (2.134 sec/step)
Train Epoch: 16 iloader:6 [20/42 (48%)] Loss:4.591530: , (2.133 sec/step)
Train Epoch: 16 iloader:6 [21/42 (50%)] Loss:4.337608: , (2.133 sec/step)
Train Epoch: 16 iloader:6 [22/42 (52%)] Loss:2.429850: , (2.133 sec/step)
Train Epoch: 16 iloader:6 [23/42 (55%)] Loss:3.385922: , (2.134 sec/step)
Train Epoch: 16 iloader:6 [24/42 (57%)

Train Epoch: 16 iloader:8 [26/42 (62%)] Loss:3.755799: , (2.130 sec/step)
Train Epoch: 16 iloader:8 [27/42 (64%)] Loss:2.987751: , (2.130 sec/step)
Train Epoch: 16 iloader:8 [28/42 (67%)] Loss:3.777104: , (2.130 sec/step)
Train Epoch: 16 iloader:8 [29/42 (69%)] Loss:4.049496: , (2.130 sec/step)
Train Epoch: 16 iloader:8 [30/42 (71%)] Loss:3.787176: , (2.131 sec/step)
Train Epoch: 16 iloader:8 [31/42 (74%)] Loss:5.966642: , (2.131 sec/step)
Train Epoch: 16 iloader:8 [32/42 (76%)] Loss:3.455058: , (2.129 sec/step)
Train Epoch: 16 iloader:8 [33/42 (79%)] Loss:3.931520: , (2.131 sec/step)
Train Epoch: 16 iloader:8 [34/42 (81%)] Loss:3.485382: , (2.130 sec/step)
Train Epoch: 16 iloader:8 [35/42 (83%)] Loss:3.874784: , (2.131 sec/step)
Train Epoch: 16 iloader:8 [36/42 (86%)] Loss:3.765640: , (2.131 sec/step)
Train Epoch: 16 iloader:8 [37/42 (88%)] Loss:3.872909: , (2.132 sec/step)
Train Epoch: 16 iloader:8 [38/42 (90%)] Loss:3.837785: , (2.132 sec/step)
Train Epoch: 16 iloader:8 [39/42 (93%)

Train Epoch: 17 iloader:1 [35/43 (81%)] Loss:3.198352: , (2.132 sec/step)
Train Epoch: 17 iloader:1 [36/43 (84%)] Loss:4.038917: , (2.131 sec/step)
Train Epoch: 17 iloader:1 [37/43 (86%)] Loss:2.771703: , (2.132 sec/step)
Train Epoch: 17 iloader:1 [38/43 (88%)] Loss:2.828915: , (2.131 sec/step)
Train Epoch: 17 iloader:1 [39/43 (91%)] Loss:4.016065: , (2.132 sec/step)
Train Epoch: 17 iloader:1 [40/43 (93%)] Loss:3.202836: , (2.132 sec/step)
Train Epoch: 17 iloader:1 [41/43 (95%)] Loss:4.027249: , (2.130 sec/step)
Train Epoch: 17 iloader:1 [42/43 (98%)] Loss:4.051560: , (2.133 sec/step)
test stored done 72.79036402702332
Train Epoch: 17 iloader:2 [0/53 (0%)] Loss:3.860593: , (2.133 sec/step)
write finish
Train Epoch: 17 iloader:2 [1/53 (2%)] Loss:3.167371: , (2.131 sec/step)
Train Epoch: 17 iloader:2 [2/53 (4%)] Loss:3.095667: , (2.131 sec/step)
Train Epoch: 17 iloader:2 [3/53 (6%)] Loss:4.032001: , (2.131 sec/step)
Train Epoch: 17 iloader:2 [4/53 (8%)] Loss:4.529576: , (2.130 sec/step)
